In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi':180})

import sys
sys.path.append("/home/msdos/DESI-QA/desiarc-main/arc")
sys.path.append("/home/msdos/DESI-QA/")
sys.path.append("/home/msdos/DESI-QA/output/figures/")
import find_center as fc
from spotfinder import spotfinder
from xylib import calc_movetables as cm
import reportlib as rl

figPath = '/home/msdos/DESI-QA/output/figures/LinPhiTests/10ArcThetaTest/'

In [2]:
testStart = np.datetime64('2023-06-30 10:22:00')
testFinish = testStart + np.timedelta64(1,"h")

# Setting paths
datapath = "../output/database.csv"
fidpath = "../output/fiddb.csv"
df = rl.importToDf(datapath,fidpath,testStart,testFinish)

/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/msdos/DESI-QA/reportlib.py:144: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/home/ms

In [7]:
df.columns

Index(['label', 'session', 'mtang1', 'mtang2', 'move', 'posid', 'direction',
       'speed', 'motor', 'angle', 'xpix', 'ypix', 'x_mm', 'y_mm', 'xytgt',
       'peaks', 'fwhm', 'fidx0', 'fidy0', 'fidx1', 'fidy1', 'fidx2', 'fidy2',
       'fidx3', 'fidy3', 'pix2mm', 'sigpix2mm', 'ArcSession', 'xc2mm', 'yc2mm',
       'R2mm', 'xc2pix', 'yc2pix', 'MountConfiguration', 'Alpha',
       'RequestedMove', 'ObservedMove', 'MeanAlpha', 'StdAlpha'],
      dtype='object')